# 0.1 추천 시스템 복습


##  추천 시스템의 세 가지 유형

### 1 내용 기반 필터링 (Content-Based Filtering, CBF)

- 콘텐츠의 **유사도(similarity)** 를 기반으로 사용자에게 추천
- 사용자가 이용하거나 선택한 아이템과 유사한 아이템 추천
- **예시:** 특정 유저가 ‘SF + 액션’ 장르를 좋아한다면 → ‘SF 액션’ 영화 추천



### 2 협업 필터링 (Collaborative Filtering)

- 서비스 내 **다수의 사용자 선호도 정보**를 기반으로 추천  
- **취향이 비슷한 사용자들이 있다면, A의 행동을 B에게도 추천할 수 있다**

#### 협업 필터링의 두 방식

1. **사용자 기반 (User-Based Filtering)**  
   → 나와 **취향이 비슷한 사용자**들이 좋아한 아이템 추천

2. **아이템 기반 (Item-Based Filtering)**  
   → 내가 좋아한 아이템과 **유사한 다른 아이템** 추천



### CBF vs CF 비교

| 구분 | 설명 |
|------|------|
| **콘텐츠 기반 필터링 (CBF)** | - **한 사용자의 행동 이력** 기반<br>- 사용자가 좋아한 아이템들의 **속성(특징)** 유사도 계산 후 비슷한 아이템 추천<br> 예: “당신이 좋아한 제품과 유사한 제품이에요” |
| **협업 필터링 (CF)** | - **여러 사용자의 행동/평가 데이터를 활용**하여 추천<br> User-based: 나와 **비슷한 사용자**가 좋아한 아이템 추천<br>Item-based: 내가 선호한 **아이템과 유사한 아이템** 추천 |


###  하이브리드 필터링 (Hybrid Filtering)

- 둘 이상의 **추천 알고리즘을 조합**한 방식  
- CBF와 CF의 단점을 보완하기 위해 등장한 혼합 방식



# 협업 필터링(CF)의 구현 방식

협업 필터링은 구현 방식에 따라 크게 **기억 기반(Memory-Based)** 과  
**모델 기반(Model-Based)** 으로 나뉨



## 1 기억 기반 (Memory-Based) 협업 필터링

기억 기반 CF는 실제 평점 데이터인 **User–Item 행렬**을 메모리에 저장하고  
그 위에서 **유사도(similarity)** 를 계산하여 추천합니다.

### 1. 사용자 기반 협업 필터링 (User-Based CF, UBCF)

- **아이디어:** 나와 **취향이 비슷한 사용자**를 찾는다  
- **작동 원리:**  
  1. 나와 다른 모든 사용자 간의 **유사도 계산** (코사인 유사도 등)  
  2. 유사도가 높은 **상위 k명 이웃** 선정  
  3. 이웃들이 좋아하지만 나는 보지 않은 아이템 추천  
- **문제점:**  
  - 사용자가 많아질수록 연산량이 급증 (확장성 문제)  
  - 취향 다양성 때문에 유사도 계산이 불안정함


### 2. 아이템 기반 협업 필터링 (Item-Based CF, IBCF)

- **아이디어:** 함께 선호되는 아이템(=공동 선택)을 찾는다
- **작동 원리:**  
  1. 모든 아이템 쌍에 대한 **유사도 계산**  
  2. 사용자가 선호한 아이템과 유사한 아이템을 추천  
- **장점:**  
  - 아이템은 사용자보다 **변화가 훨씬 적음**  
  → 유사도 행렬이 안정적이고, UBCF 대비 확장성이 뛰어남



## 2 기억 기반 협업 필터링의 한계

기억 기반 CF는 쉽고 직관적이지만, **실제 서비스 적용 시 다음과 같은 문제**가 발생

### 1. 데이터 희소성 (Sparsity)

- 대부분의 사용자는 전체 아이템 중 **아주 일부만 평가**  
- User–Item 행렬의 대부분이 비어 있음 (0이 99% 이상)  
→ 공통 평점이 적어 유사도 계산이 부정확 → 추천 품질 저하

### 2. 확장성 (Scalability)

- 사용자/아이템 수 증가 → 유사도 계산해야 할 쌍이 **기하급수적으로 증가**  
- 복잡도: **O(N²)**  
- 대규모 서비스에서는 실시간 처리가 사실상 불가능

### 3. 콜드 스타트 (Cold Start)

- 새 아이템은 평가 이력이 없으므로  
  → 어떤 유저와도 유사도 계산 불가  
  → 추천될 기회 자체가 없음



## 3 모델 기반 협업 필터링의 등장

모델 기반 CF는 기억 기반의 구조적 한계를 해결하기 위해 등장했습니다.

### 핵심 아이디어

> **평점 데이터에서 “잠재 요인(Latent Factor)”을 학습하여,  
> 이를 통해 비어 있는 평점을 예측한다.**

- 유저의 ‘취향 벡터’  
- 아이템의 ‘특성 벡터’  
이 두 벡터의 **내적(dot product)** 으로 평점을 예측하는 방식

### 장점

- 복잡한 유사도 전체 계산이 필요 없음  
- 예측은 단순한 벡터 내적으로 가능 → **확장성 문제 해결**  
- Sparse 데이터에서도 효율적





# 1. 모델 기반 추천

## 1.1 잠재 요인과 행렬 분해



### 1.1.1 잠재 요인 (Latent Factor, LF)

잠재 요인(Latent Factor)은 사용자–아이템 상호작용에 숨어 있는  
사용자의 취향 또는 아이템 특성을 설명하는 **숨겨진 차원(Hidden Dimension)**

- 사람이 직접 "이 요인은 액션이다"라고 이해할 수 있는 개념이 아님  
- 모델이 평점 패턴을 가장 잘 설명하도록 **스스로 학습한 추상 개념**

‼ 모델은 잠재 요인의 의미를 모르며,  
우리가 사후적으로 **해석하는 것일 뿐**입니다.



### 1.1.2 행렬 분해 (Matrix Factorization, MF)

MF는 사용자–아이템 평점 행렬을 사용자 취향 벡터 와 아이템 특성 벡터의 내적으로 설명한다는 가정에서 출발


- **목표:**  
  관측된 평점을 가장 잘 설명하는 쌍을 학습하는 것




### 1.1.3 MF 수행 방식 비교

| 방식 | 접근 방식 | 특징 |
|------|-----------|-------|
| **SVD** | 선형대수 기반 분해 | 결측치 처리 필요, Dense 행렬에 적합 |
| **SGD** | 최적화(경사하강법) | 희소 행렬 직접 학습 가능, 실제 CF에 적합 |
| **NMF** | 비음수 제약 조건 | 해석력 높고 직관적 의미 부여 가능 |
| **PMF** | 확률적 모델 | 예측 불확실성까지 모델링 가능 |



## SVD (Singular Value Decomposition)

### 특잇값의 의미
- 값이 클수록 더 많은 정보를 설명  
- 추천 시스템에서는 상위 k개만 사용해 차원 축소(Truncated SVD)

### Full vs Truncated SVD

| 구분 | Full SVD | Truncated SVD |
|------|-----------|----------------|
| 내용 | 모든 특잇값 사용 | 상위 k개만 사용 |
| 적합 | Dense 데이터 | Sparse 데이터(추천 시스템) |

### SVD 기반 MF 구현 단계

1. 결측치 채우기(평균 등으로 imputation)
2. SVD 계산
3. 상위 k개의 특잇값만 선택(Truncation)
4. 사용자/아이템 잠재 요인 추출 → 내적으로 평점 예측



## SGD (Stochastic Gradient Descent)

SVD의 한계를 해결하기 위해 등장:

- 희소 행렬 직접 학습 가능
- 대규모 행렬에서도 효율적




## NMF (Non-Negative Matrix Factorization)

- → 해석력이 높음 (음수 없음)

### 장점
- 잠재 요인이 **선호 강도**로 명확하게 해석됨  
  - 예: 액션 0.7, 로맨스 0.3

### 학습 방법
- 곱셈 업데이트 규칙(Multiplicative Update Rules) 사용


## PMF (Probabilistic Matrix Factorization)

- 행렬 분해를 확률 모델링 관점에서 접근
- 예측값뿐만 아니라 불확실성(분산/표준편차)까지 반환 가능
- 베이지안 MF 모델의 기반

예:  
- 기존 MF → “평점 4.2점 예상”  
- PMF → “평점 4.2점, 표준편차 0.3”


# 2. 최근 추천시스템의 방향성

추천시스템은 여러 비즈니스 요구와 맞물려 빠르게 발전하고 있으며, 딥러닝의 성능 향상에 따라 다양한 모델링 기법이 등장하고 있음

## 2.1. VAE 기반 협업 필터링 (Variational Autoencoder CF)

### 개념
- 사용자의 아이템 상호작용 벡터(예: 구매/시청 이력)를 입력으로 받아, 잠재 공간에서 이를 재구성하도록 학습하는 방식
- **인코더**는 사용자 이력을 잠재변수 분포로 압축하고  
  **디코더**는 이를 다시 아이템 분포 형태로 복원



### 장점
- 단일 벡터가 아닌 **확률적 잠재 분포**로 사용자 선호를 모델링 → **불확실성 반영**
- 희소한 상호작용 데이터에서도 강건
- 대표 모델: **Mult-VAE**


### 2.1.1 MF의 한계와 딥러닝의 등장

MF는 유용하지만 다음과 같은 한계가 존재

| MF의 한계 | 설명 | 딥러닝의 해결 방식 |
| --- | --- | --- |
| **선형성의 한계** | 예측이 단순 내적 기반 | 복잡한 패턴을 비선형 신경망으로 학습 |
| **불확실성 무시** | 사용자 취향을 단일 벡터로만 표현 | VAE는 **확률적 분포**로 모델링 |

→ 이 한계를 해결하기 위해 등장한 접근법 중 하나가 **VAE 기반 CF**.



### 2.1.2 왜 AutoEncoder(AE)가 아닌 VAE인가?

AE는 단일 지점(Point Estimate)만 예측하기 때문에  
데이터가 희소할 경우 노이즈에 매우 취약

예시:  
- **AE**: “사용자 A의 취향은 1개의 벡터!”  
- **VAE**: “사용자 A는 평균 µ, 분산 σ의 **분포**를 가진다!”

즉, VAE는 데이터 부족에 따른 **불확실성(uncertainty)**까지 모델링

| 구분 | AE | VAE |
| --- | --- | --- |
| 잠재 표현 | 단일 값 | **분포** |
| 불확실성 반영 | 불가 | 가능 |
| 희소 데이터 | 취약 | **강건** |



### 2.1.3 VAE 기반 CF (Mult-VAE)의 작동 구조

| 구성 요소 | 역할 |
| --- | --- |
| 입력 \(x\) | 사용자 아이템 상호작용 벡터 |
| 인코더 | \(x \rightarrow z\) 분포로 매핑 |
| 잠재 변수 \(z\) | 샘플링된 사용자 취향 벡터 |
| 디코더 | \(z \rightarrow\) 아이템 분포 복원 |



### 2.1.4 VAE의 핵심: ELBO 목적 함수 & KL Divergence


#### 1) 재구성 오차
- 원본 벡터를 얼마나 정확히 복원했는지 평가(AE 역할과 동일)

#### 2) KL Divergence
- 잠재 분포가 표준 정규분포 근처에 있도록 강제  
- 잠재 공간을 **부드럽고 매끄럽게(Smooth)** 유지  
- 과적합 및 불규칙한 잠재 분포를 방지

#### 비유
| 모델 | 잠재 공간 | 희소 데이터 처리 |
| --- | --- | --- |
| AE | 혼돈의 섬처럼 흩어짐 | 보간 어려움 → 취약 |
| VAE | 매끄럽게 정규화된 고속도로 | 안정적 보간 가능 |

→ 이 때문에 VAE는 희소 데이터에서 매우 강력



### VAE-CF의 주요 장점 요약

1. **확률적 잠재 분포 모델링 → 불확실성 반영**
2. **희소한 데이터에 강건**
3. **딥러닝 기반 비선형 표현 학습**


## 2.2. GNN 기반 사용자 선호 예측

### 2.2.0 GNN의 필요성

VAE 같은 잠재요인 모델은  
- “관계”,  
- “연결 구조”,  
- “사용자 간 유사성의 전파”  

와 같은 네트워크 구조 정보를 직접적으로 활용하지 못함

→ **이 관계 자체를 데이터로 보고 직접 학습할 수 없을까?**  
그래서 등장한 것이 **GNN(Graph Neural Network)**



## 2.2.1 GNN (Graph Neural Network)


GNN은 사용자-아이템 상호작용을  
단순한 점들의 집합이 아니라, 거대한 관계망(Graph)으로 바라봄



### 그래프란?

- **노드(Node)**: 개체(사용자, 아이템 등)  
- **엣지(Edge)**: 관계(본다, 구매한다, 좋아요 등)



### 그래프를 사용하는 이유

1. **관계 데이터를 자연스럽게 표현**
   - “친구 관계”, “소비 관계” 같은 추상적 개념을  
     노드-엣지 형태로 명시적 구조화 가능

2. **Non-Euclidean 구조 표현 학습 가능**
   - 이미지·텍스트처럼 격자(Grid) 구조가 아님  
   - 복잡하고 불규칙한 현실 세계 데이터를 가장 자연스럽게 표현


### 핵심 개념
사용자는 자신이 본 아이템과 연결되고,  
아이템은 자신을 본 사용자들과 연결

따라서  
- “나와 비슷한 사용자”는 **나의 이웃의 이웃(2-hop neighbor)**  
- GNN은 이런 관계 기반 전파(Message Passing)를 통해  
  사용자 취향을 정교하게 예측




### 2.2.2 GNN의 구조

##  GNN 작동 구조

1. **그래프 구조 생성**  
   - 노드는 개별 객체  
   - 엣지는 객체 간 관계나 연결성 의미  

2. **메시지 패싱(Message Passing)**  
   1. **메시지 생성(Message)**  
   2. **집계(Aggregation)**  
   3. **갱신(Update)**  

3. **최종 출력**


##  GNN 핵심 아이디어

**이웃 노드의 정보를 받아 나의 임베딩을 업데이트한다 (Message Passing).**


그래프를 입력받아 메시지 패싱을 수행하며, 각 노드는 **이웃 노드의 잠재 요인 정보를 수집**하여 자신의 임베딩을 업데이트 
이 과정을 K번 반복하여 더 먼 이웃의 정보까지 반영

## 1. 그래프 구조 생성

Netflix 추천 시스템 예시를 통해 설명합니다.

| 사용자 | 사용자 특징 | 영화 | 영화 특징 | 평점 |
| --- | --- | --- | --- | --- |
| U1 | 20대 | A | 액션, 평균 평점 4.3점 | 5 |
| U1 | . | B | 스릴러, 평균 평점 4.1점 | 4 |
| U2 | 30대 | A | . | 3 |
| U2 | . | C | SF, 평균 평점 4.8점 | 5 |
| U3 | 20대 | C | . | 4 |

→ 노드: U1, U2, U3, A, B, C  
→ 엣지: 사용자-아이템 간 상호작용 (예: U1 → A 평점 5)


### (1) 인접 행렬 (Adjacency Matrix)

노드 간 연결 정보:


- 6×6 행렬  
- U1–A 연결 → 1  
- U2–U1 연결 없음 → 0  

|  | U1 | U2 | U3 | A | B | C |
|---|---|---|---|---|---|---|
| U1 | 0 | 0 | 0 | 1 | 1 | 0 |
| U2 | 0 | 0 | 0 | 1 | 0 | 1 |
| U3 | 0 | 0 | 0 | 0 | 0 | 1 |
| A | 1 | 1 | 0 | 0 | 0 | 0 |
| B | 1 | 0 | 0 | 0 | 0 | 0 |
| C | 0 | 1 | 1 | 0 | 0 | 0 |



### (2) 피처 행렬 (Feature Matrix)



|  | 20대 | 30대 | 액션 | SF | 평균평점 4.5↑ |
|---|---|---|---|---|---|
| U1 | 1 | 0 | 0 | 0 | 0 |
| U2 | 0 | 1 | 0 | 0 | 0 |
| U3 | 1 | 0 | 0 | 0 | 0 |
| A  | 0 | 0 | 1 | 0 | 0 |
| B  | 0 | 0 | 0 | 0 | 0 |
| C  | 0 | 0 | 0 | 1 | 1 |



## 2. 메시지 패싱(Message Passing)

GNN의 핵심 단계는 다음 3단계:

1. **메시지 생성 (Message)**
2. **이웃 메시지 집계 (Aggregation)**
3. **노드 상태 갱신 (Update)**  


→ 노드 A → B → C 로 순서대로 메시지가 전달됨  
→ 이웃에게서 받은 정보를 기반으로 새로운 값 생성



###  [1] 메시지 생성 (Message)


###  [2] 집계 (Aggregation)



여러 이웃 메시지를 요약:

- sum  
- mean  
- max  
- attention-weighted sum  
  등의 방식 사용



###  [3] 갱신 (Update)

→ 이 과정을 L번 반복하면 L-hop 이웃 정보까지 반영 가능  



###  GNN vs Transformer

| 구분 | GNN | Transformer |
|---|---|---|
| 메시지 계산 | 이웃 임베딩 전달 | Q/K/V 생성 |
| 집계 | AGG 함수 사용 | Attention 가중합 |
| 갱신 | 이웃 기반 업데이트 | Context 벡터 생성 |


### 2.2.3 LightGCN (Light Graph Convolutional Networks)

LightGCN은 추천 시스템에 특화된 GNN의 경량화 버전



## 주요 GNN 아키텍처

1. **GCN** – 그래프 Convolution  
2. **GRN** – RNN + 그래프  
3. **GAT** – Attention 기반 그래프  
4. **GraphSAGE** – 샘플링 기반, concat 방식  

## LightGCN


LightGCN은 **오직 정규화된 이웃 임베딩의 합(Propagation)** 만 사용

### LightGCN이 제거한 것

1. **특징 변환(Feature Transformation) 제거**  
   → 레이어마다의 특징 삭제

2. **비선형 활성화 함수 제거**  
   → ReLU 등 사용 X

### LightGCN의 핵심

- 복잡한 연산 제거  
- 관계(Edge) 중심 추천 모델의 본질 강화  
- 계산 효율 상승  
- 대규모 추천 시스템에 특히 적합  



## 2.3. Transformer 기반 추천 시스템
: **순서와 문맥으로 사용자를 이해하다**

## 1. 기존 추천 시스템의 순차적 한계

Transformer 도입 전, CF와 GNN 모델은 **순차적 정보(Sequence)** 를 다루는 데 명확한 한계를 갖고 있었습니다.

### 1.1 CF의 순차적 한계

- **순서 무시:**  
  예)  
  `[신발 → 바지 → 셔츠]`  
  `[셔츠 → 바지 → 신발]`  
  두 사용자를 동일한 패턴으로 인식함.
- **문맥 인식 부족:**  
  ‘방금’ 구매한 아이템과 ‘1년 전’ 구매한 아이템의 중요도를 구분하기 어려움
- **다음 행동 예측의 어려움:**  
  사용자의 ‘다음 행동’보다는 전체적인 ‘선호도’ 파악에 집중한 모델 구조

### 1.2 GNN의 순차적 한계

- **지역 정보 전파의 한계:**  
  GNN은 Message Passing을 기반으로 하여 여러 단계를 거칠수록 정보가 희석됨
- **장기 의존성(Long-range dependency) 문제:**  
  Self-Attention처럼 먼 과거의 행동을 바로 참조하기 어려움.  
  GNN은 그래프 구조(이웃)에 의존하고, Transformer는 전체 시퀀스를 한 번에 고려할 수 있음

> **결론:**  
> CF / GNN은 일반적 선호도 분석에는 탁월하나,  
> **순서·문맥·장기 의존성 기반의 ‘동적 의도’ 분석에는 한계**가 있음 → Transformer 등장 배경



## 2. Transformer: 사용자 행동을 순서(Sequence)로 이해하다

### 문맥이 중요한 이유: ‘의도의 파악’

Transformer는 **Self-Attention**을 통해 시퀀스 내 문맥과 핵심 의도를 포착한다.


### 예시 1. 동일 아이템이라도 ‘문맥’에 따라 의도가 달라진다

#### 🔹 상황: 사용자가 **등산화**를 클릭

#### 문맥 A – 캠핑 준비
- 행동 순서:  
  `텐트 → 침낭 → 등산화`
- Transformer 해석:  
  등산화 = “**캠핑 장비**”
- 추천 결과:  
  “코펠”, “백패킹 배낭”, “랜턴”

#### 문맥 B – 겨울 출퇴근 준비
- 행동 순서:  
  `롱패딩 → 방한 장갑 → 등산화`
- Transformer 해석:  
  등산화 = “**방한 신발**”
- 추천 결과:  
  “발열 내의”, “울 양말”, “넥워머”

> 동일 아이템이라도 문맥(A/B)에 따라 ‘완전히 다른 의도’로 해석됨.


### 예시 2. 멀리 떨어진 과거 행동이 핵심 의도가 될 때  
(**Long-range dependency**)

#### 행동 시퀀스:
1. **레이저 프린터 구매**
2. 가습기 검색
3. 강아지 사료 클릭  
   … (20개 행동)
4. **토너 검색**

#### Self-Attention 해석
- “토너”는 시퀀스 전체를 바라보고 가장 관련 높은 **프린터**와 높은 Attention 형성
- 추천:  
  “프린터 전용 토너”, “A4 용지”

> Transformer는 거리와 관계없이 ‘의도와 가장 관련 있는 행동’을 찾아냄.



## 3. Transformer의 핵심: Self-Attention

### Self-Attention = 시퀀스 내 모든 아이템 관계를 한 번에 계산

- “3일 전 행동이 방금 클릭보다 더 중요할 수 있다”는 직관을 모델이 직접 반영.
- 장기 의존성을 자연스럽게 해결.

### 핵심 구성: Query, Key, Value

| 역할 | 의미 |
|------|------|
| **Query(Q)** | 현재 아이템이 찾고 싶은 정보 |
| **Key(K)** | 다른 아이템들이 갖는 ‘색인’ 정보 |
| **Value(V)** | 실제 정보(아이템 의미 벡터) |

### 연산 과정

1. `Q · K` → **관련성 점수 계산**
2. Softmax → **가중치 생성**
3. 가중치 × V → **가중 평균**

→ 결과: 관련성이 높은 정보는 크게, 낮은 정보는 적게 반영된 **문맥 벡터 생성**



## 4. Transformer 기반 주요 추천 모델



## 4.1 SASRec (Self-Attentive Sequential Recommendation)

- **방식:** 단방향 어텐션 (GPT 구조)
- **특징:**  
  - 미래 정보 차단 (Masked Self-Attention)  
  - ‘다음 아이템 예측’에 최적화  
- **문제점:**  
  - 계산량 많음 (시퀀스 길이만큼 loss 계산)  
  - 단방향성 → 미래 문맥 활용 불가

→ **이 한계를 보완한 모델: BERT4Rec**


## 4.2 BERT4Rec (BERT for Recommendation)

- **방식:** 양방향(Bi-directional) Self-Attention
- **훈련 방법:** 시퀀스 중 일부를 `[MASK]`로 가리고 원래 아이템을 예측  
- **특징:**  
  - 과거+미래 문맥 모두 활용  
  - 정보 누수 방지를 위해 Masking 적용  
- **목표:**  
  - 시퀀스 내 가려진 아이템 복원 (Masked Item Prediction)



## SASRec vs. BERT4Rec 비교

### **성능 요약**

- **상호작용 많은 데이터셋** → **BERT4Rec 우세**  
  (더 넓은 문맥 이해 능력)
- **상호작용 적은 데이터셋** → **SASRec 우세**  
  (학습 구조 단순, 과적합 적음)




## 진짜 최신 트렌드 : LLM의 시대

트랜스포머가 시퀀셜 추천의 문을 연 것은 맞지만, 이제는 **LLM(Large Language Models)**이 추천 패러다임을 바꾸고 있음



###  관점의 전환: 추천을 `랭킹` 문제가 아닌 `언어` 문제로 본다

- **과거 추천 방식**
  - 조합별로 미리 학습된 공식에 따라 점수 출력
  - 예: (사용자 A, 아이템 123) → 0.8점

- **LLM 기반 추천**
  - 점수 계산이 아닌 **문장 생성**으로 추천을 수행
  - ID가 아닌 **의미(Semantic)** 기반 추천
  - 텍스트·이미지·오디오 등 **멀티모달 데이터 활용**
  - 예:"사용자 A는 [아이템 1, 2, 3]을 좋아했습니다. 이 사용자가 좋아할 다음 아이템은 [아이템 4]입니다."

 **단순 점수가 아닌 자연스러운 이유 설명과 서술형 추천 가능**


###  LLM이 열어준 새로운 가능성

LLM은 단순 메타데이터가 아니라 **설명·리뷰·스토리·분위기 등 텍스트 전체를 이해**할 수 있음

####  대화형 추천  
사용자의 자연어 요구를 직접 이해  
예: *“어제 본 영화랑 비슷한데, 분위기는 더 밝은 거 없어?”*

####  Zero-shot 추천  
학습 데이터에 없던 새로운 아이템도  
아이템의 **텍스트 설명만으로** 추천 가능  

 **Cold-Start 문제 해결의 강력한 솔루션**

> **Cold-Start 문제**  
> 새로운 사용자 또는 아이템에 대한 정보가 부족해 추천이 어렵던 전통적 문제
> → LLM은 텍스트 이해 능력으로 이를 크게 완화



###  P5 / TALLRec

#### **P5**  
모든 추천 업무(랭킹, 다음 아이템 예측, 설명 생성 등)를  **하나의 통합 텍스트 생성 문제**로 재정의한 모델



#### **TALLRec**  
잘 만들어진 LLM(LLaMA 등)을 추천 작업에 맞게 **효율적으로 튜닝하는 프레임워크**.



 LLM은 단순 성능 향상이 아니라**추천 시스템의 작동 방식 자체를 바꾸고 있음.**



## 현업의 고민: 책임감 있는 AI

모델 성능(Accuracy)만으로는 부족하다.


### 1 인기 편향(Popularity Bias)

추천 시스템이 인기 아이템만 과도하게 노출하는 현상.

- 현실 데이터는 인기 아이템에 집중되어 있음  
- 모델 학습 시 이 편향이 그대로 강화됨  
- “Rich-get-richer” 현상 발생  
- 비인기·신규 아이템은 거의 추천되지 않음

**예시**
- 유튜브: 상위 1% 영상이 조회수 80%
- 이커머스: 베스트셀러 중심 노출

사용자는 새로운 아이템을 발견할 기회를 잃고 신규 아이템은 **성장할 기회조차 없음**


### 2 설명가능성 (Explainable AI, XAI)

- “왜 이 추천을 했는가?”를 설명할 수 있어야 함  
- GNN, Transformer 같은 딥러닝 모델은 **블랙박스 문제** 존재  
- 고객·비즈니스 담당자를 설득할 수 있는 투명한 추천 로직 필요

 **편향과 공정성을 고려하며 설명 가능한 모델 = 현업에서 가장 중요한 가치**


## 전부 딥러닝을 사용하지 않는 이유

###  현실적인 문제: 비용

- 딥러닝 모델 → **GPU 필수**, 연산비용 큼  
- 모든 아이템을 실시간 스코어링? → **불가능에 가까움**  
- 대규모 서비스는 서버비 폭발 → 지속 불가

그래서 실제 산업에서는 다음과 같이 운영함

## 실전 구조: 선형 모델 + 딥러닝/LLM **하이브리드 전략**


###  1단계: 후보군 생성 (Candidate Generation)

- **목표**: 수백만 개 중에서 "가능성 있는 수백 개"를 매우 빠르게 추리기
- **사용 모델**: 선형 모델
  - 계산 속도 빠름
  - 비용 효율적
  - 실시간 대규모 트래픽 대응 가능



###  2단계: 순위 매기기 (Ranking)

- **목표**: 후보군 수백 개를 정교하게 정렬
- **사용 모델**: 딥러닝(GNN, Transformer, VAE 등)
  - 비선형 패턴 학습
  - 순서·문맥·상호작용 반영
  - 높은 정확도 도출

 **현업은 “빠름 + 정교함”을 위해 두 가지 모델을 함께 사용**


